# NPA : Neural News Recommendation with Personalized Attention (2019)
https://jenn1won.tistory.com/19?category=1046828

- NPA는 content-based 뉴스 추천 방법이다.
- CNN을 이용해 뉴스 표현을 학습하고, 유저들이 클릭한 뉴스로부터 유저 표현을 학습한다.
- word-level personalized attention으로 서로 다른 유저에 대한 중요 단어에 집중한다.
- news-level personalized attention으로 서로 다른 유저에 대한 뉴스 클릭에 집중한다.

In [1]:
import sys
import os
import numpy as np
import zipfile
from tqdm import tqdm
import scrapbook as sb
from tempfile import TemporaryDirectory
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.models.deeprec.deeprec_utils import download_deeprec_resources 
from recommenders.models.newsrec.newsrec_utils import prepare_hparams
from recommenders.models.newsrec.models.npa import NPAModel
from recommenders.models.newsrec.io.mind_iterator import MINDIterator
from recommenders.models.newsrec.newsrec_utils import get_mind_data_set

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))


System version: 3.7.13 (default, Mar 29 2022, 02:18:16) 
[GCC 7.5.0]
Tensorflow version: 2.7.3


In [3]:
tmpdir = TemporaryDirectory()
data_path = tmpdir.name

train_news_file = os.path.join(data_path, 'train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'train', r'behaviors.tsv')
valid_news_file = os.path.join(data_path, 'valid', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'valid', r'behaviors.tsv')
wordEmb_file = os.path.join(data_path, "utils", "embedding.npy")
userDict_file = os.path.join(data_path, "utils", "uid2index.pkl")
wordDict_file = os.path.join(data_path, "utils", "word_dict.pkl")
yaml_file = os.path.join(data_path, "utils", r'npa.yaml')

mind_url, mind_train_dataset, mind_dev_dataset, mind_utils = get_mind_data_set('demo')

if not os.path.exists(train_news_file):
    download_deeprec_resources(mind_url, os.path.join(data_path, 'train'), mind_train_dataset)
    
if not os.path.exists(valid_news_file):
    download_deeprec_resources(mind_url, os.path.join(data_path, 'valid'), mind_dev_dataset)
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/newsrec/', os.path.join(data_path, 'utils'), mind_utils)

100%|█████████████████████████████████████| 17.0k/17.0k [00:08<00:00, 2.05kKB/s]
100%|█████████████████████████████████████| 9.84k/9.84k [00:04<00:00, 2.30kKB/s]
100%|█████████████████████████████████████| 95.0k/95.0k [00:26<00:00, 3.55kKB/s]


In [4]:
# Create Hyper-parameters
hparams = prepare_hparams(yaml_file, wordEmb_file=wordEmb_file,
                         wordDict_file=wordDict_file,
                         userDict_file=userDict_file,
                         batch_size=32,
                         epochs=5)
print(hparams)

HParams object with values {'support_quick_scoring': False, 'dropout': 0.2, 'attention_hidden_dim': 200, 'head_num': 4, 'head_dim': 100, 'filter_num': 400, 'window_size': 3, 'vert_emb_dim': 100, 'subvert_emb_dim': 100, 'gru_unit': 400, 'type': 'ini', 'user_emb_dim': 100, 'learning_rate': 0.0001, 'optimizer': 'adam', 'epochs': 5, 'batch_size': 32, 'show_step': 100000, 'title_size': 10, 'his_size': 50, 'data_format': 'news', 'npratio': 4, 'metrics': ['group_auc', 'mean_mrr', 'ndcg@5;10'], 'word_emb_dim': 300, 'cnn_activation': 'relu', 'model_type': 'npa', 'loss': 'cross_entropy_loss', 'wordEmb_file': '/tmp/tmpxo2yo8ez/utils/embedding.npy', 'wordDict_file': '/tmp/tmpxo2yo8ez/utils/word_dict.pkl', 'userDict_file': '/tmp/tmpxo2yo8ez/utils/uid2index.pkl'}


In [5]:
iterator = MINDIterator

In [6]:
# Train
model = NPAModel(hparams, iterator, seed=42)
print(model.run_eval(valid_news_file, valid_behaviors_file))

2022-07-11 14:51:05.456662: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-11 14:51:05.518485: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-11 14:51:05.522068: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-11 14:51:05.522347: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

{'group_auc': 0.5229, 'mean_mrr': 0.2328, 'ndcg@5': 0.2376, 'ndcg@10': 0.303}


In [7]:
%%time
model.fit(train_news_file, train_behaviors_file, valid_news_file, valid_behaviors_file)

0it [00:00, ?it/s]2022-07-11 14:57:29.764210: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.44GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-07-11 14:57:29.764260: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.44GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-07-11 14:57:30.012450: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.65GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-07-11 14:57:30.012506: W tensorflow/core/common_runtime/bfc_alloc

at epoch 1
train info: logloss loss:1.5236308640537999
eval info: group_auc:0.5719, mean_mrr:0.2487, ndcg@10:0.3322, ndcg@5:0.2671
at epoch 1 , train time: 101.1 eval time: 102.0


1086it [01:38, 11.04it/s]
8874it [01:38, 90.37it/s]


at epoch 2
train info: logloss loss:1.4091489926447087
eval info: group_auc:0.5934, mean_mrr:0.2669, ndcg@10:0.3535, ndcg@5:0.2902
at epoch 2 , train time: 98.4 eval time: 102.1


1086it [01:38, 11.04it/s]
8874it [01:39, 89.56it/s]


at epoch 3
train info: logloss loss:1.349556202717249
eval info: group_auc:0.5946, mean_mrr:0.2699, ndcg@10:0.358, ndcg@5:0.2926
at epoch 3 , train time: 98.4 eval time: 103.0


1086it [01:37, 11.09it/s]
8874it [01:36, 91.82it/s]


at epoch 4
train info: logloss loss:1.2976336348561732
eval info: group_auc:0.5938, mean_mrr:0.2687, ndcg@10:0.3558, ndcg@5:0.2907
at epoch 4 , train time: 98.0 eval time: 100.5


1086it [01:37, 11.11it/s]
8874it [01:36, 91.86it/s]


at epoch 5
train info: logloss loss:1.2633124786955656
eval info: group_auc:0.5958, mean_mrr:0.2698, ndcg@10:0.3564, ndcg@5:0.2915
at epoch 5 , train time: 97.7 eval time: 100.5
CPU times: user 8min 29s, sys: 20.3 s, total: 8min 49s
Wall time: 16min 41s


In [10]:
%%time
res_syn = model.run_eval(valid_news_file, valid_behaviors_file)
print(res_syn)

8874it [01:38, 90.41it/s]


{'group_auc': 0.5958, 'mean_mrr': 0.2698, 'ndcg@5': 0.2915, 'ndcg@10': 0.3564}
CPU times: user 40.2 s, sys: 2.73 s, total: 42.9 s
Wall time: 1min 42s


In [11]:
sb.glue('res_syn', res_syn)

In [ ]:
# Save the model
model_path = os.path.join(data_path, "model")
os.makedirs(model_path, exist_ok=True)

model.model.save_weights(os.path.join(model_path, "npa_ckpt"))

In [12]:
# Output prediction File
group_impr_indexes, group_labels, group_preds = model.run_slow_eval(valid_news_file, valid_behaviors_file)

with open(os.path.join(data_path, 'prediction.txt'), 'w') as f:
    for impr_index, preds in tqdm(zip(group_impr_indexes, group_preds)):
        impr_index += 1
        pred_rank = (np.argsort(np.argsort(preds)[::-1]) + 1).tolist()
        pred_rank = '[' + ','.join([str(i) for i in pred_rank]) + ']'
        f.write(' '.join([str(impr_index), pred_rank])+ '\n')

f = zipfile.ZipFile(os.path.join(data_path, 'prediction.zip'), 'w', zipfile.ZIP_DEFLATED)
f.write(os.path.join(data_path, 'prediction.txt'), arcname='prediction.txt')
f.close()

8874it [01:38, 90.48it/s]
7538it [00:00, 70042.43it/s]
